<a href="https://colab.research.google.com/github/aso1901103/AI_tech2020/blob/master/2020AI0305_fishvideo_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 魚の画像を学習して動画解析に利用する
魚の画像をダウンロードして解凍

In [1]:
# 画像のzipのダウンロード
import urllib.request as req
# 魚が映っている画像
url = "https://github.com/masatokg/sample_photo/raw/master/fish.zip"
save_file = "fish.zip"
req.urlretrieve(url, save_file)

#　魚が映っていない画像
url = "https://github.com/masatokg/sample_photo/raw/master/nofish.zip"
save_file = "nofish.zip"
req.urlretrieve(url, save_file)

# それぞれ解凍する
# -q ←quietモードでメッセージを表示せず解凍
!unzip -o -q "./fish.zip"
!unzip -o -q "./nofish.zip"

### 魚の学習のための定義

In [2]:
# ライブラリのimport
import cv2, os, glob, pickle
# データ分割するやつ
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
# 学習アルゴリズム
from sklearn.ensemble import RandomForestClassifier
# 正解度を表す
from sklearn.metrics import accuracy_score

In [3]:
# 画像のサイズやパスを指定
image_size = (64, 32)
path = "/content"
path_fish = path + "/fish"
path_nofish = path + "/nofish"
x = [] # 画像データ(説明変数)用配列
y = [] # ラベルデータ(目的変数)用配列
print(path_fish)
print(path_nofish)

/content/fish
/content/nofish


### 学習に使う自作関数を定義

In [4]:
# 画像データを読み込んで説明変数、目的変数に追加する処理の関数を定義
def read_dir(path, label, x, y, image_size):
  # 正規表現にマッチするファイルパスリストを取得する
  files = glob.glob(path + "/*.jpg")
  print(files)
  # ファイルパスリストの要素1つずつループ
  for f in files:
    img = cv2.imread(f) # 画像データを読み込む
    img = cv2.resize( img, image_size) # 学習用にサイズを統一
    img_data = img.reshape( -1,) # 行数任意の一次元配列に変換
    x.append(img_data) # 画像データを説明変数配列に追加
    y.append(label) # 画像データと同じ順番位置でラベル値を目的変数に追加

### 学習処理

In [5]:
# 画像データを読み込む
read_dir(path_nofish, 0, x, y, image_size) # ラベルを0:Falseとして登録
read_dir(path_fish, 1, x, y, image_size) # ラベルを1:Trueとして登録

# データを学習用とテスト検証用に分割する
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# ランダムフォレストアルゴリズムで学習モデルを生成、学習する
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

# 精度の確認
y_pred = clf.predict(x_test)
print("学習精度：", accuracy_score(y_test, y_pred))

# 学習モデルのメモリ状態をファイル保持
# バイナリファイルを書き込み用に開く
with open("fish.pkl", "wb") as fp:
  pickle.dump(clf, fp)

['/content/nofish/3450.jpg', '/content/nofish/254.jpg', '/content/nofish/3631.jpg', '/content/nofish/3511.jpg', '/content/nofish/3652.jpg', '/content/nofish/3632.jpg', '/content/nofish/3468.jpg', '/content/nofish/23.jpg', '/content/nofish/13.jpg', '/content/nofish/3647.jpg', '/content/nofish/3456.jpg', '/content/nofish/33.jpg', '/content/nofish/3656.jpg', '/content/nofish/22.jpg', '/content/nofish/3444.jpg', '/content/nofish/3473.jpg', '/content/nofish/132.jpg', '/content/nofish/124.jpg', '/content/nofish/28.jpg', '/content/nofish/275.jpg', '/content/nofish/31.jpg', '/content/nofish/39.jpg', '/content/nofish/3580.jpg', '/content/nofish/186.jpg', '/content/nofish/27.jpg', '/content/nofish/278.jpg', '/content/nofish/0.jpg', '/content/nofish/257.jpg', '/content/nofish/253.jpg', '/content/nofish/3453.jpg', '/content/nofish/29.jpg', '/content/nofish/3643.jpg', '/content/nofish/10.jpg', '/content/nofish/11.jpg', '/content/nofish/3507.jpg', '/content/nofish/3648.jpg', '/content/nofish/279.jpg

### 学習モデルを利用して動画から魚の画像を検出
解析の準備

In [6]:
import cv2, os, copy, pickle
from google.colab.patches import cv2_imshow

# 学習済みモデルファイルをプログラムに読み込む
# バイナリファイルを読み込み用に開く
with open("fish.pkl", "rb" ) as fp:
  clf = pickle.load(fp)

# 解析画像出力ディレクトリ用文字列
output_dir = "./bestshot"
# 解析画像出力ディレクトリが存在しなければ作成
if not os.path.isdir(output_dir):
  os.mkdir(output_dir)
# 以下でも同じことができる
# os.makedirs(output_dir, exist_ok=True)

# 各所利用変数を定義
img_last = None # 前回の画像データ用変数
fish_th = 3 # 画像を出力するかどうかの閾値(一枚あたり魚の検出数)
count = 0
frame_count = 0

### 動画ファイルの読み込み処理


In [7]:
url = "https://github.com/masatokg/sample_photo/raw/master/fish.mp4"
save_file = "fish.mp4"
req.urlretrieve(url, save_file)

('fish.mp4', <http.client.HTTPMessage at 0x7f68b89078d0>)

### 動画をプログラムに読み込み、フレームの差分により変化を検出、検出した部分に魚がいるかを推論して、魚がいると判定したら該当フレームをbestshotディレクトリにファイル保存

In [8]:
# openCVで度がデータを読み込み
cap = cv2.VideoCapture(save_file)
# 読み込むフレームがなくなるまで無限ループ
while True:
  is_ok, frame = cap.read()
  if not is_ok:
    print("end")
    break
  frame = cv2.resize(frame, (640, 360)) # 差分検出対象フレームのサイズを統一
  # 検出エリア枠表示用の画像データを作る
  frame2 = copy.copy(frame)
  frame_count += 1
  # 前のフレームと比較するためにグレースケール、さらにブラックアンドホワイトに変換
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  # ガウスぼかしを入れる
  gray = cv2.GaussianBlur(gray, (15, 15), 0)
  img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
  # 前の画像がない(最初の画像)でなければ実行
  # img_last ←前の画像を保存しておく変数
  if not img_last is None:
    # 差分を得る
    frame_diff = cv2.absdiff(img_last, img_b)
    # 差分が見つかった場所情報のリストを取得
    cnts = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    # 各差分領域に魚が映っているか一つずつ調べる
    fish_count = 0
    for pt in cnts:
      # 検出した領域情報を変数に格納
      x, y, w, h = cv2.boundingRect(pt)
      if w<100 or w>500:
        continue # ノイズとして無視
      # 抽出した差分領域に魚が映っているか推論するために画像サイズを揃える
      imgex = frame[y:y+h, x:x+w]
      imgex = cv2.resize(imgex, (64, 32)) # 学習モデルのサイズにあわせる
      image_data = imgex.reshape( -1) # 推論のために一次元配列に変換
      # 推論
      pred_y = clf.predict([image_data])
      # 魚と判定できたら領域に枠をつけてframe2という画像にする
      # 配列の0番目に結果が入っている
      if pred_y[0] == 1:
        fish_count += 1
        # 枠を描く
        cv2.rectangle(frame2, (x,y), (x+w,y+h), (0,255,0), 2) # Greenの枠
    # 閾値以上に魚が映っていなければ、画像ファイルとして出力
    if fish_count > fish_th:
      fname = output_dir + "/fish" + str(count) + ".jpg"
      cv2.imwrite(fname, frame) # 指定した画像名でフレームを保存
      count += 1 # 画像ファイルカウンタを+1
  # 負荷が重いのでコメントアウト
  # cv2.imshow( frame2) # 枠付きの画像を表示する
  img_last = img_b # 前回のフレーム画像を記憶
  
cap.release() # 動画ファイルを開放
# 出力した枚数を表示
print("ok", count, "/", frame_count, "/枚検出" )

end
ok 97 / 1989 /枚検出
